In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns', None)
import datetime
import catboost
from catboost import CatBoostClassifier,Pool
from time import time
from tqdm import tqdm_notebook as tqdm
import os
import random
import itertools
import json
import pprint
import gc
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold,GroupKFold
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import cohen_kappa_score
from collections import Counter

In [3]:
submission = pd.read_csv('/kaggle/input/data-science-bowl-2019/sample_submission.csv')

In [ ]:
if(len(submission['installation_id'])>=1000):
    submission.to_csv('submission.csv',index=False)
    exit(0)

In [4]:
def qwk(a1, a2, max_rat=3):
    assert(len(a1) == len(a2))
    a1 = np.asarray(a1, dtype=int)
    a2 = np.asarray(a2, dtype=int)

    hist1 = np.zeros((max_rat + 1, ))
    hist2 = np.zeros((max_rat + 1, ))

    o = 0
    for k in range(a1.shape[0]):
        i, j = a1[k], a2[k]
        hist1[i] += 1
        hist2[j] += 1
        o +=  (i - j) * (i - j)

    e = 0
    for i in range(max_rat + 1):
        for j in range(max_rat + 1):
            e += hist1[i] * hist2[j] * (i - j) * (i - j)

    e = e / a1.shape[0]

    return 1 - o / e


In [5]:
def spec(value,*args):
    i= specs[specs['event_id'] == value].index.values[-1]
    print('Index :',i)
    print('Event_code :',train[train['event_id'] == value]['event_code'].unique()[-1])
    for arg in args:
        if(arg == 'info'):
         print(specs[arg][i])
        elif(arg == 'args'):
         print(pprint.pprint(json.loads(specs[arg][i])))
        else:
         print('Nothing')

In [6]:
def event(value):
    i = train[train['event_id'] == value].index.values[-1]
    print(pprint.pprint(json.loads(train['event_data'][i])))

In [7]:
def get_game_feature(df,data):
    g0_d = dict(df[df['type'] == 'Game'].groupby('installation_id')['type'].value_counts())
    g0 = {}
    for (k,i),v in g0_d.items():
        g0[k] = v
    del g0_d
    df['G0'] = df['installation_id'].map(g0)
    df['G0'].fillna(0,inplace=True)

    g1_d = dict(df[(df['event_code'] == 4020) & (df['type'] == 'Game')].groupby('installation_id')['event_code'].value_counts())
    g1 = {}
    for (k,i),v in g1_d.items():
        g1[k] = v
    del g1_d
    df['G1'] = df['installation_id'].map(g1)
    df['G1'].fillna(0,inplace=True)
    del g1
    
    df['contains_true'] = df[(df['event_code'] == 4020) & (df['type'] == 'Game')]['event_data'].map(lambda x:1 if(str(x).find('"correct":true')>= 0) else 0)
    g2 = dict(df[df['event_code'] == 4020].groupby('installation_id')['contains_true'].sum())
    df['G2'] = df['installation_id'].map(g2)
    df['G2'].fillna(0,inplace=True)
    for c in ['contains_true']:
        df.pop(c)
        
    df['G3'] = df['G2'] / df['G1']
    df['G3'] = df['G3'].map(lambda x:3 if(x == 1.0) else(2 if((x >= 0.5) & (x <1.0)) else (1 if((x>0.0) &  (x<0.5)) else 0)))
    df['G3'].fillna(0,inplace=True)
    
    i = 4
    for d in tqdm(data):
        g_attempt_d = dict(df[(df['type'] == 'Game') & (df['event_code'] == 4020) & (df['title'] == d)].groupby('installation_id')['title'].value_counts())
        g_attempt = {}
        for (k,z),v in g_attempt_d.items():
            g_attempt[k] = v

        df[f'G{i}_attempt'] = df['installation_id'].map(g_attempt)
        df[f'G{i}_attempt'].fillna(0,inplace=True)

        df['contains_true'] = df[(df['type'] == 'Game') & (df['event_code'] == 4020) & (df['title'] == d)]['event_data'].map(lambda x:1 if(str(x).find('"correct":true')>=0) else 0)
        g = dict(df.groupby('installation_id')['contains_true'].sum())
        df[f'G{i}_correct'] = df['installation_id'].map(g)
        df[f'G{i}_correct'].fillna(0,inplace=True)

        df[f'G{i}_accuracy'] = df[f'G{i}_correct'] / df[f'G{i}_attempt']
        df[f'G{i}_accuracy'] = df[f'G{i}_accuracy'].map(lambda x:3 if(x == 1.0) else(2 if((x >= 0.5) & (x <1.0)) else (1 if((x>0.0) &  (x<0.5)) else 0)))
        df[f'G{i}_accuracy'].fillna(0,inplace=True)

        for c in ['contains_true',f'G{i}_attempt']:
            df.pop(c)
            
        i = i + 1

In [8]:
def get_attempted_assessment_results(df,data):
    if(data == 'train'):
        df = df[(df['type'] == 'Assessment') & (((df['event_code'] == 4100) & (df['title'] != 'Bird Measurer (Assessment)')) | ((df['event_code'] == 4110) & (df['title'] == 'Bird Measurer (Assessment)')))]
    else:
        df =  df[(df['type'] == 'Assessment') &  ((df['event_count'] == 1) | ((df['event_code'] == 4100) & (df['title'] != 'Bird Measurer (Assessment)')) | ((df['event_code'] == 4110) & (df['title'] == 'Bird Measurer (Assessment)')))]
    session_count = df['game_session'].value_counts().to_dict()
    df['assessment_attempt_count'] = df['game_session'].map(session_count)

    df['contains_true_assessment'] = df['event_data'].map(lambda x: True if (x.find('"correct":true')>=0) else False)

    change_value = {
        True : 1,
        False : 0
    }
    df['contains_true_assessment'] = df['contains_true_assessment'].map(change_value)

    correct_attempt = dict(df.groupby('game_session',sort=False)['contains_true_assessment'].sum())
    df['contains_true_assessment_count'] = df['game_session'].map(correct_attempt)

    for c in ['contains_true_assessment']:
        df.pop(c)

    df['accumulated_accuracy'] = np.where((df['contains_true_assessment_count'] == 0),0,(df['contains_true_assessment_count']/df['assessment_attempt_count']))

    df.loc[(df['type'] == 'Assessment'), 'accuracy_group'] = 0
    df.loc[(df['accumulated_accuracy'] == 1) & (df['type'] == 'Assessment'), 'accuracy_group'] = 3
    df.loc[(df['accumulated_accuracy'] == 0.5) & (df['type'] == 'Assessment'), 'accuracy_group'] = 2
    df.loc[(df['accumulated_accuracy'] < 0.5) & (df['accumulated_accuracy'] > 0) & (df['assessment_attempt_count'] > 0) & (df['type'] == 'Assessment'), 'accuracy_group'] = 1

    df.rename(columns = {'contains_true_assessment_count': 'num_correct',
                            'accumulated_accuracy':'accuracy',
                            'assessment_attempt_count': 'total_attempt'},inplace=True)
    df = df.drop_duplicates(subset = 'game_session',keep = 'last')

In [9]:
def get_validation_data(df,num_samples,random_state):
    validation_data = df[(df['type'] == 'Assessment') & (((df['event_code'] == 4100) & (df['title'] != 'Bird Measurer (Assessment)')) | ((df['event_code'] == 4110) & (df['title'] == 'Bird Measurer (Assessment)')))]
    validation_data.drop_duplicates(subset = 'game_session',keep = 'last',inplace=True)
    if(isinstance(num_samples,float)):
            validation_data = validation_data.sample(frac = num_samples,random_state = random_state)
            print(validation_data.shape)
    else:
            validation_data = validation_data.sample(n = num_samples,random_state = random_state)
            print(validation_data.shape)
    return validation_data

In [10]:
def get_features(df):
    total_features = []
    clip_features = ['num_clip_watched']
    game_features = ['num_unique_games','total_game_actions','total_duration_spend_for_game','total_game_attempts','total_correct_attempts','last_game_played_accuracy','gaming_accuracy']
    activity_features = []
    total_data = {}
    total_features = clip_features + game_features + activity_features
    for i,install_id in tqdm(df.groupby('installation_id',sort=False)):

        clip_data = {eve : 0 for eve in clip_features}
        game_data = {eve : 0 for eve in game_features}
        activity_data = {}
        event_code_count = {eve : 0 for eve in list_of_event_code}

        for j,session in install_id.groupby('game_session',sort=False):

            session_type = session['type'].iloc[0]
            session_title = session['title'].iloc[0]

            if((session_type == 'Clip')):
                clip_data['num_clip_watched'] += 1

            elif(session_type == 'Game'):
                game_data['num_unique_games'] += 1
                game_data['total_game_actions'] += session['event_code'].count()
                game_data['total_duration_spend_for_game'] += int(session['game_time'].iloc[-1]/1000)
                game_data['total_game_attempts'] += session[session['event_code'] == 4020]['event_code'].count()
                game_data['total_correct_attempts'] += session[session['event_code'] == 4020]['event_data'].map(lambda x:1 if(str(x).find('"correct":true') >= 0) else 0).sum()
                game_data['gaming_accuracy'] += round((game_data['total_correct_attempts']/game_data['total_game_attempts']),3) if(game_data['total_game_attempts']>0) else 0
                game_data['last_game_played_accuracy'] = round(game_data['total_correct_attempts'] / game_data['total_game_attempts'],3) if(game_data['total_game_attempts']>0) else 0
            elif(session_type == 'Activity'):
                pass
            elif(session_type == 'Assessment'):
                #game_preprocessing


                #Activity preprocessing


                #Clip preprocessing



                total_data[j] = {}
                total_data[j].update(clip_data)
                total_data[j].update(activity_data)
                total_data[j].update(game_data)

                clip_data = {eve : 0 for eve in clip_features}
                game_data = {eve : 0 for eve in game_features}
                activity_data = {}
    return total_features,total_data

In [11]:
def convert_dict_to_list(d):
    for i,j in d.items():
        d[i] = [v for k,v in d[i].items()]
    return d

In [12]:
test = pd.read_csv('/kaggle/input/data-science-bowl-2019/test.csv')
train = pd.read_csv('/kaggle/input/data-science-bowl-2019/train.csv')

In [13]:
train['timestamp'] = pd.to_datetime(train['timestamp'])
test['timestamp'] = pd.to_datetime(test['timestamp'])

In [14]:
train_labels = pd.read_csv('/kaggle/input/data-science-bowl-2019/train_labels.csv')
specs = pd.read_csv('/kaggle/input/data-science-bowl-2019/specs.csv')

In [15]:
assessment_id = list(train[train['type'] == 'Assessment']['installation_id'].unique())
train = train.loc[train['installation_id'].isin(assessment_id)]

In [16]:
train = train.reset_index(drop=False)
train.drop(columns = ['index'],axis = 1, inplace = True)

In [17]:
list_of_event_code = set(train['event_code'].unique()).union(set(test['event_code'].unique()))
list_of_event_code = list(list_of_event_code)

In [ ]:
train_game = train[(train['type'] == 'Game') | (train['type'] == 'Assessment')]
train_game = train_game.reset_index(drop=False)
train_game.drop(columns = ['index'],axis = 1, inplace = True)

train_activity = train[(train['type'] == 'Assessment') | (train['type'] == 'Activity')]
train_activity = train_activity.reset_index(drop=False)
train_activity.drop(columns = ['index'],axis = 1, inplace = True)

train_clip = train[(train['type'] == 'Assessment') | (train['type'] == 'Clip')]
train_clip = train_clip.reset_index(drop=False)
train_clip.drop(columns = ['index'],axis = 1, inplace = True)

In [18]:
column_mapper = {'index': 'game_session',0:'num_clip_watched',1:'num_unique_games',2:'total_game_actions',3:'total_duration_spend_for_game',4:'total_game_attempts',5:'total_correct_attempts',6:'last_game_played_accuracy',7:'gaming_accuracy'}

**Train**

In [19]:
features, train_dict = get_features(train)

In [20]:
for i,j in tqdm(train_dict.items()):
    train_dict[i] = [v for k,v in train_dict[i].items()]

In [21]:
train = train[(train['type'] == 'Assessment')].drop_duplicates(subset='game_session',keep = 'first').reset_index(drop=False).drop(columns=['index'],axis=1)

In [22]:
train_merge = pd.DataFrame().from_dict(train_dict)
train_merge = train_merge.T.reset_index(drop=False).rename(columns = column_mapper)

In [34]:
train.head()

,game_session,game_time,title,world,num_clip_watched,num_unique_games,total_game_actions,total_duration_spend_for_game,total_game_attempts,total_correct_attempts,last_game_played_accuracy,gaming_accuracy
0,901acc108f55a5a1,0,3,2,11.0,4.0,284.0,278.0,44.0,28.0,0.636,2.915
1,77b8ee947eb84b4e,0,4,2,3.0,2.0,241.0,271.0,29.0,7.0,0.241,0.433
2,6bdf9623adc94d89,0,3,2,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000
3,e7e7db2a241eadcc,0,3,2,10.0,4.0,320.0,389.0,26.0,23.0,0.885,3.588
4,9501794defd84e4d,0,3,2,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000


In [35]:
train_merge.head()

,game_session,num_clip_watched,num_unique_games,total_game_actions,total_duration_spend_for_game,total_game_attempts,total_correct_attempts,last_game_played_accuracy,gaming_accuracy
0,901acc108f55a5a1,11.0,4.0,284.0,278.0,44.0,28.0,0.636,2.915
1,77b8ee947eb84b4e,3.0,2.0,241.0,271.0,29.0,7.0,0.241,0.433
2,6bdf9623adc94d89,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000
3,e7e7db2a241eadcc,10.0,4.0,320.0,389.0,26.0,23.0,0.885,3.588
4,9501794defd84e4d,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000


In [23]:
train = train.merge(train_merge,on='game_session',how='left')

In [24]:
for c in ['installation_id']:
    train.pop(c)

**Test**

In [25]:
features, test_dict = get_features(test)

In [26]:
for i,j in tqdm(test_dict.items()):
    test_dict[i] = [v for k,v in test_dict[i].items()]

In [27]:
test = test[(test['type'] == 'Assessment')].drop_duplicates(subset='game_session',keep = 'first').reset_index(drop=False).drop(columns=['index'],axis=1)

In [28]:
test_merge = pd.DataFrame().from_dict(test_dict)
test_merge = test_merge.T.reset_index(drop=False).rename(columns = column_mapper)

In [29]:
test = test.merge(test_merge,on='game_session',how='left')

In [30]:
test.head(10)

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world,num_clip_watched,num_unique_games,total_game_actions,total_duration_spend_for_game,total_game_attempts,total_correct_attempts,last_game_played_accuracy,gaming_accuracy
0,7ad3efc6,8b38fc0d2fd315dc,2019-09-11 18:56:11.918000+00:00,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",00abaee7,1,2000,0,Cart Balancer (Assessment),Assessment,CRYSTALCAVES,13.0,2.0,238.0,2095.0,30.0,19.0,0.633,1.280
1,90d848e0,348d7f09f96af313,2019-09-12 13:52:12.193000+00:00,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",00abaee7,1,2000,0,Cauldron Filler (Assessment),Assessment,MAGMAPEAK,1.0,1.0,135.0,188.0,13.0,10.0,0.769,0.769
2,90d848e0,009c890ce6c4f3e3,2019-09-22 21:23:06.296000+00:00,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",01242218,1,2000,0,Cauldron Filler (Assessment),Assessment,MAGMAPEAK,4.0,4.0,367.0,572.0,34.0,33.0,0.971,3.933
3,7ad3efc6,e8e62de939f916bc,2019-09-22 21:38:58.329000+00:00,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",01242218,1,2000,0,Cart Balancer (Assessment),Assessment,CRYSTALCAVES,6.0,2.0,138.0,215.0,14.0,13.0,0.929,1.838
4,5b49460a,ab61cae5e3215355,2019-09-22 21:50:37.983000+00:00,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",01242218,1,2000,0,Chest Sorter (Assessment),Assessment,CRYSTALCAVES,2.0,2.0,164.0,196.0,18.0,18.0,1.000,2.000
5,3bfd1a65,31423dbcd717919e,2019-09-22 22:14:10.921000+00:00,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",01242218,1,2000,0,Mushroom Sorter (Assessment),Assessment,TREETOPCITY,10.0,1.0,56.0,64.0,12.0,12.0,1.000,1.000
6,f56e0afc,597a8839a5a3468d,2019-09-22 22:27:47.533000+00:00,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",01242218,1,2000,0,Bird Measurer (Assessment),Assessment,TREETOPCITY,3.0,2.0,302.0,292.0,19.0,14.0,0.737,1.570
7,7ad3efc6,1fef5d54cb4b775a,2019-10-09 20:23:16.209000+00:00,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",01242218,1,2000,0,Cart Balancer (Assessment),Assessment,CRYSTALCAVES,4.0,1.0,62.0,76.0,3.0,3.0,1.000,1.000
8,3bfd1a65,4b165a330a0bdd6c,2019-09-21 11:28:21.757000+00:00,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",017c5718,1,2000,0,Mushroom Sorter (Assessment),Assessment,TREETOPCITY,6.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000
9,3bfd1a65,be0b655ad1fee30c,2019-07-27 16:28:10.394000+00:00,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",01a44906,1,2000,0,Mushroom Sorter (Assessment),Assessment,TREETOPCITY,10.0,1.0,78.0,77.0,16.0,12.0,0.750,0.750


**Splitting train data**

In [31]:
for c in tqdm(['event_id','event_code','event_data','event_count','type','timestamp']):
    for df in [train,test]:
        df.pop(c)

In [32]:
list_of_user_activities = list(set(train['title'].value_counts().index).union(set(test['title'].value_counts().index)))
activities_map = dict(zip(list_of_user_activities, np.arange(len(list_of_user_activities))))

train['title'] = train['title'].map(activities_map)
test['title'] = test['title'].map(activities_map)
train_labels['title'] = train_labels['title'].map(activities_map)

In [33]:
world = list(set(train['world'].unique().tolist()).union(set(test['world'].unique().tolist())))
world = dict(zip(world,np.arange(len(world))))

train['world'] = train['world'].map(world)
test['world'] = test['world'].map(world)

In [ ]:
x = train.drop(columns=['accuracy_group'],axis=1)
y = train['accuracy_group']

In [ ]:
def make_classifier():
    clf = CatBoostClassifier(
                               loss_function='MultiClass',
                               task_type="CPU",
                               learning_rate=0.01,
                               iterations=100,
                               od_type="Iter",
                               early_stopping_rounds=50,
                               random_seed=2019,
                               colsample_bylevel=0.87,
                               eval_metric='Kappa',
                              )
        
    return clf
oof = np.zeros(len(x))

In [ ]:
oof = np.zeros(len(x))
NFOLDS = 5
folds = KFold(n_splits=NFOLDS, shuffle=True, random_state=2019)


for fold, (trn_idx, test_idx) in enumerate(folds.split(x, y)):
    
    print(f'Training on fold {fold+1}')
    clf = make_classifier()
    clf.fit(x.loc[trn_idx], y.loc[trn_idx], eval_set=(x.loc[test_idx], y.loc[test_idx]),
                          use_best_model=True, verbose=500)
    
    oof[test_idx] = clf.predict(x.loc[test_idx]).reshape(len(test_idx))
    print('OOF QWK:', qwk(y, oof))
    
print('-' * 30)
print('OOF QWK:', qwk(y, oof))
print('-' * 30)

**Garbage**

In [ ]:
cols = train.columns.tolist()
cols.remove('installation_id')
for i ,data in train.groupby('installation_id',sort=False):
    new_data = data[cols]
    new_data['installation_id'] = i
    break
new_data.head()

In [ ]:
def get_features(df:pd.DataFrame(),which):
    
    new_df_list_values = []
    event_code_count_dict = {eve : 0 for eve in list_of_event_code}
    new_df_dict = {}
        
    df = df.reset_index(drop= False)
    df.drop(columns=['index'],axis=1,inplace=True)
    
    for i,data in tqdm(df.groupby('installation_id',sort=False)):
        print(i)
        new_data = data[df.columns.tolist()]
        new_data['installation_id'] = i
        
        new_data = new_data.reset_index(drop= False)
        new_data.drop(columns=['index'],axis=1,inplace=True)
        
        a1 = new_data[new_data['type'] == 'Assessment'].drop_duplicates(subset = 'game_session',keep = 'first').index.tolist()
        a2 = new_data[new_data['type'] == 'Assessment'].drop_duplicates(subset = 'game_session',keep = 'last').index.tolist()

        a = []
        a.append(0)
        print(a2)
        for i in range(len(a2)):
            a.append(a2[i])
            a.append(a2[i]+1)
        a.pop(len(a)-1)
        print(a)

        for i in range(0,len(a),2):
            print(a[i] , 'to' , a[i+1])
            new_df = data.iloc[a[i]:a[i+1]+1,:]
            g_session = new_df[new_df['type'] == 'Assessment']['game_session'].unique()[-1]
            print(g_session)
            new_df_dict[g_session] = {}
            new_df_dict[g_session]['num_game_session'] = new_df['game_session'].nunique()
            new_df_dict[g_session]['num_event_id'] = new_df['event_id'].nunique()
            new_df_dict[g_session]['num_actions_before_assessment'] = len(new_df['event_code'])
            new_df_dict[g_session][f'num_unique_{which}'] = new_df[new_df['type'] == which]['title'].nunique()
            if(which == 'Game'):
                new_df_dict[g_session]['game_attempts'] = new_df[new_df['event_code'] == 4020]['event_code'].count()
                new_df_dict[g_session]['mean_game_time'] = new_df[new_df['type'] == 'Game']['game_time'].mean()
                new_df_dict[g_session]['max_game_time'] = new_df[new_df['type'] == 'Game']['game_time'].max()
                new_df_dict[g_session]['min_game_time'] = new_df[new_df['type'] == 'Game']['game_time'].min()
                new_df_dict[g_session]['std_game_time'] = new_df[new_df['type'] == 'Game']['game_time'].std()
                new_df['gaming_accuracy'] = new_df[(new_df['type'] == 'Game') & (new_df['event_code'] == 4020)]['event_data'].map(lambda x:1 if(str(x).find('"correct":true')) else 0)
                new_df['gaming_accuracy'].fillna(0,inplace=True)
                new_df_dict[g_session]['gaming_accuracy'] = new_df['gaming_accuracy'].sum()/new_df_dict[g_session]['game_attempts']
            elif(which == 'Activity'):
                new_df_dict[g_session]['Activity actions'] = new_df[new_df['type'] == 'Activity']['event_id'].count()

            for k,v in event_code_count_dict.items():
                new_df_dict[g_session][k] = v

            for i in event_code_count_dict.keys():
                 new_df_dict[g_session][i] = new_df[new_df['event_code'] == i]['event_code'].count()
            #break
    return new_df_dict
        
o        

In [ ]:
def get_features(df:pd.DataFrame(),which):
    
    new_df_list_values = []
    event_code_count_dict = {}
    new_df_dict = {}
    
    for i in train['event_code'].unique().tolist():
        event_code_count_dict[i] = 0
        
    df = df.reset_index(drop= False)
    df.drop(columns=['index'],axis=1,inplace=True)
    
    a1 = df[df['type'] == 'Assessment'].drop_duplicates(subset = 'game_session',keep = 'first').index.tolist()
    a2 = df[df['type'] == 'Assessment'].drop_duplicates(subset = 'game_session',keep = 'last').index.tolist()
    
    a = []
    a.append(0)
    for i in range(len(a2)):
        a.append(a2[i])
        a.append(a2[i]+1)
    a.pop(len(a)-1)
    
    for i in tqdm(range(len(a))):
        new_df = df.iloc[a[i]:a[i+1],:]
        ids  = new_df['installation_id'].unique().tolist()[-1]
        new_df_dict[ids] = {}
        new_df_dict['num_game_session'] = new_df['game_session'].nunique()
        new_df_dict['num_event_id'] = new_df['event_id'].nunique()
        new_df_dict['num_actions_before_assessment'] = len(new_df['event_code'])
        new_df_dict[f'num_unique_{which}'] = new_df[new_df['type'] == which]['title'].nunique()
        if(which == 'Game'):
            new_df_dict['game_attempts'] = new_df[new_df['event_code'] == 4020]['event_code'].count()
            new_df_dict['mean_game_time'] = new_df[new_df['type'] == 'Game']['game_time'].mean()
            new_df_dict['max_game_time'] = new_df[new_df['type'] == 'Game']['game_time'].max()
            new_df_dict['min_game_time'] = new_df[new_df['type'] == 'Game']['game_time'].min()
            new_df_dict['std_game_time'] = new_df[new_df['type'] == 'Game']['game_time'].std()
            new_df['gaming_accuracy'] = new_df[(new_df['type'] == 'Game') & (new_df['event_code'] == 4020)]['event_data'].map(lambda x:1 if(str(x).find('"correct":true')) else 0)
            new_df['gaming_accuracy'].fillna(0,inplace=True)
            new_df_dict['gaming_accuracy'] = new_df['gaming_accuracy'].sum()/new_df_dict['game_attempts']
            #new_df['time_gap_before_assessment'] = new_df['']
        elif(which == 'Activity'):
            new_df_dict['Activity actions'] = new_df[new_df['type'] == 'Activity']['event_id'].count()
        
        event_code_count_dict.update(new_df_dict)
        break
    print(event_code_count_dict)

In [ ]:
def get_features(df:pd.DataFrame(),which):
    
    new_df_list_values = []
    event_code_count_dict = {eve : 0 for eve in list_of_event_code}
    new_df_dict = {}
        
    df = df.reset_index(drop= False)
    df.drop(columns=['index'],axis=1,inplace=True)
    
    a1 = df[df['type'] == 'Assessment'].drop_duplicates(subset = 'game_session',keep = 'first').index.tolist()
    a2 = df[df['type'] == 'Assessment'].drop_duplicates(subset = 'game_session',keep = 'last').index.tolist()
    
    a = []
    a.append(0)
    for i in range(len(a2)):
        a.append(a2[i])
        a.append(a2[i]+1)
    a.pop(len(a)-1)
    
    for i in tqdm(range(10)):
        new_df = df.iloc[a[i]:a[i+1],:]
        ids  = new_df['installation_id'].unique().tolist()[-1]
        g_session = new_df[new_df['type'] == 'Assessment']['game_session'].unique()[-1]
        print(ids,g_session)
        new_df_dict[ids] = {}
        new_df_dict[ids][g_session] = {}
        new_df_dict[ids][g_session]['num_game_session'] = new_df['game_session'].nunique()
        new_df_dict[ids][g_session]['num_event_id'] = new_df['event_id'].nunique()
        new_df_dict[ids][g_session]['num_actions_before_assessment'] = len(new_df['event_code'])
        new_df_dict[ids][g_session][f'num_unique_{which}'] = new_df[new_df['type'] == which]['title'].nunique()
        if(which == 'Game'):
            new_df_dict[ids][g_session]['game_attempts'] = new_df[new_df['event_code'] == 4020]['event_code'].count()
            new_df_dict[ids][g_session]['mean_game_time'] = new_df[new_df['type'] == 'Game']['game_time'].mean()
            new_df_dict[ids][g_session]['max_game_time'] = new_df[new_df['type'] == 'Game']['game_time'].max()
            new_df_dict[ids][g_session]['min_game_time'] = new_df[new_df['type'] == 'Game']['game_time'].min()
            new_df_dict[ids][g_session]['std_game_time'] = new_df[new_df['type'] == 'Game']['game_time'].std()
            new_df['gaming_accuracy'] = new_df[(new_df['type'] == 'Game') & (new_df['event_code'] == 4020)]['event_data'].map(lambda x:1 if(str(x).find('"correct":true')) else 0)
            new_df['gaming_accuracy'].fillna(0,inplace=True)
            new_df_dict[ids][g_session]['gaming_accuracy'] = new_df['gaming_accuracy'].sum()/new_df_dict[ids][g_session]['game_attempts']
        elif(which == 'Activity'):
            new_df_dict[ids][g_session]['Activity actions'] = new_df[new_df['type'] == 'Activity']['event_id'].count()
        
        for k,v in event_code_count_dict.items():
            new_df_dict[ids][g_session][k] = v
        
        for i in event_code_count_dict.keys():
             new_df_dict[ids][g_session][i] = new_df[new_df['event_code'] == i]['event_code'].count()
        #break
    return new_df_dict
        
        

In [ ]:
train_column_values = {}
for i in tqdm(train['installation_id'].unique().tolist()):
    df = train[(train['installation_id'] == i) & ((train['type'] == 'Game') | (train['type'] == 'Assessment'))]
    column_values_instance = get_features(df,'Game')
    train_column_values.update(column_values_instance)
train_column_values

In [ ]:
per_install_id_features = {}
event_code_count_per_id = {}
for i,data in tqdm(train.groupby('installation_id',sort = False)):
    new_data = data[data.columns.tolist()]
    per_install_id_features[i] = {}
    for j in ['Game','Activity','Clip']:
        per_install_id_features[i][f'num_{j}'] = new_data[new_data['type'] == j]['title'].nunique()
    per_install_id_features[i]['total_num_actions'] = new_data[(new_data['type'] == 'Game') | (new_data['type'] == 'Activity') | (new_data['type'] == 'Clip')]['event_id'].count()
    per_install_id_features[i]['total_game_attempts'] = new_data[(new_data['type'] == 'Game') & (new_data['event_code'] == 4020)]['event_code'].count()
    new_data['correct_attempts'] = new_data[(new_data['type'] == 'Game') & (new_data['event_code'] == 4020)]['event_data'].map(lambda x:1 if(str(x).find('"correct":true'))>=0 else 0)
    per_install_id_features[i]['total_correct_game_attempts'] = new_data['correct_attempts'].sum()
    per_install_id_features[i]['total_accuracy'] = per_install_id_features[i]['total_correct_game_attempts']/per_install_id_features[i]['total_game_attempts'] if per_install_id_features[i]['total_game_attempts'] >0 else 0
    
per_install_id_features

In [ ]:
clip_features = ['num_clip_watched']
game_features = ['num_unique_games','total_game_actions','total_duration_spend_for_game','total_game_attempts','total_correct_attempts','last_game_played_accuracy','gaming_accuracy']
activity_features = []
total_data = {}

for i,install_id in tqdm(test.groupby('installation_id',sort=False)):
    
    clip_data = {eve : 0 for eve in clip_features}
    game_data = {eve : 0 for eve in game_features}
    activity_data = {}
    event_code_count = {eve : 0 for eve in list_of_event_code}
    
    for j,session in install_id.groupby('game_session',sort=False):
        
        session_type = session['type'].iloc[0]
        session_title = session['title'].iloc[0]
        
        if((session_type == 'Clip')):
            clip_data['num_clip_watched'] += 1
            
        elif(session_type == 'Game'):
            game_data['num_unique_games'] += 1
            game_data['total_game_actions'] += session['event_code'].count()
            game_data['total_duration_spend_for_game'] += int(session['game_time'].iloc[-1]/1000)
            game_data['total_game_attempts'] += session[session['event_code'] == 4020]['event_code'].count()
            game_data['total_correct_attempts'] += session[session['event_code'] == 4020]['event_data'].map(lambda x:1 if(str(x).find('"correct":true') >= 0) else 0).sum()
            game_data['gaming_accuracy'] += np.round((game_data['total_correct_attempts']/game_data['total_game_attempts']),3)
            game_data['last_game_played_accuracy'] = np.round(game_data['total_correct_attempts'] / game_data['total_game_attempts'],3) if(game_data['total_game_attempts']>0) else 0
        elif(session_type == 'Activity'):
            pass
        elif(session_type == 'Assessment'):
            #game_preprocessing
            
            
            #Activity preprocessing
            
            
            #Clip preprocessing
            
            
            
            total_data[j] = {}
            total_data[j].update(clip_data)
            total_data[j].update(activity_data)
            total_data[j].update(game_data)
            
            clip_data = {eve : 0 for eve in clip_features}
            game_data = {eve : 0 for eve in game_features}
            activity_data = {}
    #break   
            
total_data

In [ ]:
train = train[(train['type'] == 'Assessment') & (((train['event_code'] == 4100) & (train['title'] != 'Bird Measurer (Assessment)')) | ((train['event_code'] == 4110) & (train['title'] == 'Bird Measurer (Assessment)')))]

session_count = train['game_session'].value_counts().to_dict()
train['assessment_attempt_count'] = train['game_session'].map(session_count)

train['contains_true_assessment'] = train['event_data'].map(lambda x: True if (x.find('"correct":true')>=0) else False)

change_value = {
    True : 1,
    False : 0
}
train['contains_true_assessment'] = train['contains_true_assessment'].map(change_value)

correct_attempt = dict(train.groupby('game_session',sort=False)['contains_true_assessment'].sum())
train['contains_true_assessment_count'] = train['game_session'].map(correct_attempt)

for c in ['contains_true_assessment']:
    train.pop(c)
    
train['accumulated_accuracy'] = np.where((train['contains_true_assessment_count'] == 0),0,(train['contains_true_assessment_count']/train['assessment_attempt_count']))

train.loc[(train['type'] == 'Assessment'), 'accuracy_group'] = 0
train.loc[(train['accumulated_accuracy'] == 1) & (train['type'] == 'Assessment'), 'accuracy_group'] = 3
train.loc[(train['accumulated_accuracy'] == 0.5) & (train['type'] == 'Assessment'), 'accuracy_group'] = 2
train.loc[(train['accumulated_accuracy'] < 0.5) & (train['accumulated_accuracy'] > 0) & (train['assessment_attempt_count'] > 0) & (train['type'] == 'Assessment'), 'accuracy_group'] = 1

train.rename(columns = {'contains_true_assessment_count': 'num_correct',
                        'accumulated_accuracy':'accuracy',
                        'assessment_attempt_count': 'total_attempt'},inplace=True)
train = train.drop_duplicates(subset = 'game_session',keep = 'last')
train = train.reset_index(drop=False)
train.drop(columns = ['index'],axis = 1, inplace = True)
train.shape

In [ ]:
test =  test[(test['type'] == 'Assessment') &  ((test['event_count'] == 1) | ((test['event_code'] == 4100) & (test['title'] != 'Bird Measurer (Assessment)')) | ((test['event_code'] == 4110) & (test['title'] == 'Bird Measurer (Assessment)')))]
session_count = test['game_session'].value_counts().to_dict()
test['assessment_attempt_count'] = test['game_session'].map(session_count)

test['contains_true_assessment'] = test['event_data'].map(lambda x: True if (x.find('"correct":true')>=0) else False)

change_value = {
    True : 1,
    False : 0
}
test['contains_true_assessment'] = test['contains_true_assessment'].map(change_value)

correct_attempt = dict(test.groupby('game_session',sort=False)['contains_true_assessment'].sum())
test['contains_true_assessment_count'] = test['game_session'].map(correct_attempt)


for c in ['contains_true_assessment']:
    test.pop(c)

test['accumulated_accuracy'] = np.where((test['contains_true_assessment_count'] == 0),0,(test['contains_true_assessment_count']/test['assessment_attempt_count']))

test.loc[(test['type'] == 'Assessment'), 'accuracy_group'] = 0
test.loc[(test['accumulated_accuracy'] == 1) & (test['type'] == 'Assessment'), 'accuracy_group'] = 3
test.loc[(test['accumulated_accuracy'] == 0.5) & (test['type'] == 'Assessment'), 'accuracy_group'] = 2
test.loc[(test['accumulated_accuracy'] < 0.5) & (test['accumulated_accuracy'] > 0) & (test['assessment_attempt_count'] > 0) & (test['type'] == 'Assessment'), 'accuracy_group'] = 1

test.rename(columns = {'contains_true_assessment_count': 'num_correct',
                        'accumulated_accuracy':'accuracy',
                        'assessment_attempt_count': 'total_attempt'},inplace=True)

test = test.drop_duplicates(subset = 'game_session',keep = 'last')
test = test.reset_index(drop=False)
test.drop(columns = ['index'],axis = 1, inplace = True)
test.shape